In [ ]:
#Libraries Required 
import tensorflow as tf
import numpy as np
import pandas as pd 
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Bidirectional, Concatenate, dot
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Set parameters
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [ ]:
# Load and preprocess data
df = pd.read_csv('training_set_rel3.tsv', sep='\t',encoding='latin')
essays = df['essay']
grades = df['domain1_score']

In [ ]:
# code to load and preprocess the essays and their corresponding grades

# Tokenize essays
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(essays)
sequences = tokenizer.texts_to_sequences(essays)

In [ ]:
# Pad sequences
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
# Split data into training, validation and test sets
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
grades = grades[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
num_test_samples = int(0.2 * data.shape[0])

X_train = data[:-num_validation_samples-num_test_samples]
y_train = grades[:-num_validation_samples-num_test_samples]
X_val = data[-num_validation_samples-num_test_samples:-num_test_samples]
y_val = grades[-num_validation_samples-num_test_samples:-num_test_samples]
X_test = data[-num_test_samples:]
y_test = grades[-num_test_samples:]

In [ ]:
# Build model
embedding_layer = Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

lstm_layer = Bidirectional(LSTM(128, return_sequences=True))
lstm_output = lstm_layer(embedded_sequences)

In [ ]:
# Attention mechanism
attn_layer = Dense(1, activation='tanh')(lstm_output)
attn_layer = tf.squeeze(attn_layer, axis=-1)
attn_layer = tf.expand_dims(tf.nn.softmax(attn_layer), axis=-1)
context_vector = attn_layer * lstm_output
context_vector = tf.reduce_sum(context_vector, axis=1)

output = Dense(1, activation='sigmoid')(context_vector)
model = Model(sequence_input, output)

In [ ]:
# Train model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

In [ ]:
# Evaluate model on test data
loss, accuracy = model.evaluate(X_test, y_test, batch_size=64)
print("Test loss: {:.4f}".format(loss))
print("Test accuracy: {:.4f}".format(accuracy))